https://codeforces.com/problemset/problem/1690/G

In [ ]:
6 2 3 7 -> 6 2 2 2
6 2 1 7 -> 6 2 1 1
6 2 1 0 -> 6 2 1 0

In [ ]:
6 5 2 2 2 2 2 -> 6 5 2 2 2 2 2
6 5 2 2 1 2 2 -> 6 5 2 2 1 1 1

In [ ]:
6 5 2 2 2 1 1 0      0 1 2 5 7

6 5 2 0 0 0 0 0      
6 5 2 1 1 1 1 0

In [ ]:
6 5 2 0 0 0
6 5 1 0 0 0

In [29]:
# https://github.com/tatyam-prime/SortedSet/blob/main/SortedMultiset.py
import math
from bisect import bisect_left, bisect_right, insort
from typing import Generic, Iterable, Iterator, TypeVar, Union, List
T = TypeVar('T')

class SortedMultiset(Generic[T]):
    BUCKET_RATIO = 50
    REBUILD_RATIO = 170

    def _build(self, a=None) -> None:
        "Evenly divide `a` into buckets."
        if a is None: a = list(self)
        size = self.size = len(a)
        bucket_size = int(math.ceil(math.sqrt(size / self.BUCKET_RATIO)))
        self.a = [a[size * i // bucket_size : size * (i + 1) // bucket_size] for i in range(bucket_size)]
    
    def __init__(self, a: Iterable[T] = []) -> None:
        "Make a new SortedMultiset from iterable. / O(N) if sorted / O(N log N)"
        a = list(a)
        if not all(a[i] <= a[i + 1] for i in range(len(a) - 1)):
            a = sorted(a)
        self._build(a)

    def __iter__(self) -> Iterator[T]:
        for i in self.a:
            for j in i: yield j

    def __reversed__(self) -> Iterator[T]:
        for i in reversed(self.a):
            for j in reversed(i): yield j
    
    def __len__(self) -> int:
        return self.size
    
    def __repr__(self) -> str:
        return "SortedMultiset" + str(self.a)
    
    def __str__(self) -> str:
        s = str(list(self))
        return "{" + s[1 : len(s) - 1] + "}"

    def _find_bucket(self, x: T) -> List[T]:
        "Find the bucket which should contain x. self must not be empty."
        for a in self.a:
            if x <= a[-1]: return a
        return a

    def __contains__(self, x: T) -> bool:
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        return i != len(a) and a[i] == x

    def count(self, x: T) -> int:
        "Count the number of x."
        return self.index_right(x) - self.index(x)

    def add(self, x: T) -> None:
        "Add an element. / O(√N)"
        if self.size == 0:
            self.a = [[x]]
            self.size = 1
            return
        a = self._find_bucket(x)
        insort(a, x)
        self.size += 1
        if len(a) > len(self.a) * self.REBUILD_RATIO:
            self._build()

    def discard(self, x: T) -> bool:
        "Remove an element and return True if removed. / O(√N)"
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i == len(a) or a[i] != x: return False
        a.pop(i)
        self.size -= 1
        if len(a) == 0: self._build()
        return True

    def lt(self, x: T) -> Union[T, None]:
        "Find the largest element < x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] < x:
                return a[bisect_left(a, x) - 1]

    def le(self, x: T) -> Union[T, None]:
        "Find the largest element <= x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] <= x:
                return a[bisect_right(a, x) - 1]

    def gt(self, x: T) -> Union[T, None]:
        "Find the smallest element > x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] > x:
                return a[bisect_right(a, x)]

    def ge(self, x: T) -> Union[T, None]:
        "Find the smallest element >= x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] >= x:
                return a[bisect_left(a, x)]
    
    def __getitem__(self, x: int) -> T:
        "Return the x-th element, or IndexError if it doesn't exist."
        if x < 0: x += self.size
        if x < 0: raise IndexError
        for a in self.a:
            if x < len(a): return a[x]
            x -= len(a)
        raise IndexError

    def index(self, x: T) -> int:
        "Count the number of elements < x."
        ans = 0
        for a in self.a:
            if a[-1] >= x:
                return ans + bisect_left(a, x)
            ans += len(a)
        return ans

    def index_right(self, x: T) -> int:
        "Count the number of elements <= x."
        ans = 0
        for a in self.a:
            if a[-1] > x:
                return ans + bisect_right(a, x)
            ans += len(a)
        return ans
    
n, m = map(int, input().split())
li = list(map(int, input().split()))

ms = SortedMultiset()
ms.add(0)
cur_start_v = li[0]
for i in range(1, n):
    if cur_start_v > li[i]:
        ms.add(i)
        cur_start_v = li[i]

result = []
print(ms)
for _ in range(m):
    k, d = map(int, input().split())
    k -= 1
    
    li[k] -= d
    
    start = 0
    end = len(ms) - 1
    index = 9999999
    while start <= end:
        mid = (start + end) // 2
        
        if ms[mid] > k:
            end = mid - 1
        else:
            index = mid
            start = mid + 1

    cur_start_i = ms[index]
    
    print(11111, cur_start_i)
    
    if li[cur_start_i] > li[k]:
        if k not in ms:
            ms.add(k)
        cur_start_i = k
        to_check = index + 2
    else:
        to_check = index + 1
        
    while to_check < len(ms):
        li_index = ms[to_check]
        if li[cur_start_i] <= li[li_index]:
            ms.discard(li_index)
        else:
            break
             
    print(k, d, ms, li)
    result.append(len(ms))
    
print(*result)

4 2
6 2 3 7
{0, 1}
3 2
11111 1
2 2 {0, 1, 2} [6, 2, 1, 7]
4 7
11111 2
3 7 {0, 1, 2, 3} [6, 2, 1, 0]
3 4


In [26]:
for v in ms:
    print(li[v])

769
514
336
173
121
168


In [24]:
ms

SortedMultiset[[0, 1, 2, 3, 11, 12]]

In [20]:
li[cur_start_i], li[ms[to_check]]

(173, 168)

In [13]:
to_check

5

In [20]:
10 13 5 2 6 -> 10 10 5 2 2
10 9 5 2 6 -> 10 9 5 2 2

SortedMultiset[[0, 1]]

In [ ]:
012 34 5678 9,10,..
0   3   5  9
3 -> 1
4 -> 1
5 -> 2
10 -> 3

In [6]:
ms = SortedMultiset()
ms.add(1)
ms.add(3)
ms.add(5)
ms.add(9)

In [13]:
ms.index(4 - 0.1)

2

In [3]:
ms

SortedMultiset[[0, 1, 3]]

In [ ]:
   6 5 5 6 5 2 (0, 1, 5)
-> 6 5 5 4 5 2 (0, 1, 3, 5)
독립-> 6 5 5 1 1 1 (0, 1, 3)

현재 포함된 구간의 시작값보다 작으면 추가해줌
그 후 그 뒤엣 인덱스의 값이 더 크면 제거해줌

In [ ]:
4 3 2 2

4 1 1 1

In [ ]:
import sys
input = lambda: sys.stdin.readline().rstrip()

# https://github.com/tatyam-prime/SortedSet/blob/main/SortedMultiset.py
import math
from bisect import bisect_left, bisect_right, insort
from typing import Generic, Iterable, Iterator, TypeVar, Union, List
T = TypeVar('T')

class SortedMultiset(Generic[T]):
    BUCKET_RATIO = 50
    REBUILD_RATIO = 170

    def _build(self, a=None) -> None:
        "Evenly divide `a` into buckets."
        if a is None: a = list(self)
        size = self.size = len(a)
        bucket_size = int(math.ceil(math.sqrt(size / self.BUCKET_RATIO)))
        self.a = [a[size * i // bucket_size : size * (i + 1) // bucket_size] for i in range(bucket_size)]
    
    def __init__(self, a: Iterable[T] = []) -> None:
        "Make a new SortedMultiset from iterable. / O(N) if sorted / O(N log N)"
        a = list(a)
        if not all(a[i] <= a[i + 1] for i in range(len(a) - 1)):
            a = sorted(a)
        self._build(a)

    def __iter__(self) -> Iterator[T]:
        for i in self.a:
            for j in i: yield j

    def __reversed__(self) -> Iterator[T]:
        for i in reversed(self.a):
            for j in reversed(i): yield j
    
    def __len__(self) -> int:
        return self.size
    
    def __repr__(self) -> str:
        return "SortedMultiset" + str(self.a)
    
    def __str__(self) -> str:
        s = str(list(self))
        return "{" + s[1 : len(s) - 1] + "}"

    def _find_bucket(self, x: T) -> List[T]:
        "Find the bucket which should contain x. self must not be empty."
        for a in self.a:
            if x <= a[-1]: return a
        return a

    def __contains__(self, x: T) -> bool:
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        return i != len(a) and a[i] == x

    def count(self, x: T) -> int:
        "Count the number of x."
        return self.index_right(x) - self.index(x)

    def add(self, x: T) -> None:
        "Add an element. / O(√N)"
        if self.size == 0:
            self.a = [[x]]
            self.size = 1
            return
        a = self._find_bucket(x)
        insort(a, x)
        self.size += 1
        if len(a) > len(self.a) * self.REBUILD_RATIO:
            self._build()

    def discard(self, x: T) -> bool:
        "Remove an element and return True if removed. / O(√N)"
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i == len(a) or a[i] != x: return False
        a.pop(i)
        self.size -= 1
        if len(a) == 0: self._build()
        return True

    def lt(self, x: T) -> Union[T, None]:
        "Find the largest element < x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] < x:
                return a[bisect_left(a, x) - 1]

    def le(self, x: T) -> Union[T, None]:
        "Find the largest element <= x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] <= x:
                return a[bisect_right(a, x) - 1]

    def gt(self, x: T) -> Union[T, None]:
        "Find the smallest element > x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] > x:
                return a[bisect_right(a, x)]

    def ge(self, x: T) -> Union[T, None]:
        "Find the smallest element >= x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] >= x:
                return a[bisect_left(a, x)]
    
    def __getitem__(self, x: int) -> T:
        "Return the x-th element, or IndexError if it doesn't exist."
        if x < 0: x += self.size
        if x < 0: raise IndexError
        for a in self.a:
            if x < len(a): return a[x]
            x -= len(a)
        raise IndexError

    def index(self, x: T) -> int:
        "Count the number of elements < x."
        ans = 0
        for a in self.a:
            if a[-1] >= x:
                return ans + bisect_left(a, x)
            ans += len(a)
        return ans

    def index_right(self, x: T) -> int:
        "Count the number of elements <= x."
        ans = 0
        for a in self.a:
            if a[-1] > x:
                return ans + bisect_right(a, x)
            ans += len(a)
        return ans
    
for _ in range(int(input())):
    input()
    n, m = map(int, input().split())
    li = list(map(int, input().split()))

    ms = SortedMultiset()
    ms.add(0)
    cur_start_v = li[0]
    for i in range(1, n):
        if cur_start_v > li[i]:
            ms.add(i)
            cur_start_v = li[i]

    result = []
    for _ in range(m):
        k, d = map(int, input().split())
        k -= 1

        li[k] -= d

        start = 0
        end = len(ms) - 1
        index = 9999999
        while start <= end:
            mid = (start + end) // 2

            if ms[mid] > k:
                end = mid - 1
            else:
                index = mid
                start = mid + 1

        cur_start_i = ms[index]

        if li[cur_start_i] > li[k]:
            if k not in ms:
                ms.add(k)
            cur_start_i = k
            to_check = index + 2
        else:
            to_check = index + 1

        while to_check < len(ms):
            li_index = ms[to_check]
            if li[cur_start_i] <= li[li_index]:
                ms.discard(li_index)
            else:
                break

        result.append(len(ms))

    print(*result)